In [14]:
import torchvision.models as models
import torch
from torchvision import transforms
from PIL import Image
from torch.nn.utils.rnn import *
from torch import nn
from torch.utils import data
from convLSTM import *
import time
import numpy as np
from sklearn.model_selection import train_test_split

In [15]:
GPU = torch.cuda.is_available()
device = torch.device("cuda" if GPU else "cpu")
cuda = torch.cuda.is_available()
GPU

True

In [16]:
torch.cuda.empty_cache()

### Data Preprocessing

In [17]:
X = []
angles = []
torques = []
speeds = []
dataType = "center"
numDataPointsWanted = 3000 #Might want to change this

transform=transforms.ToTensor()
root_path = "./"
with open(root_path + "labels.txt") as f:
    start = time.time()
    i = 0
    for line in f:
        [filename, radians, torque, speed] = line.split(' ')
        
        if filename[0:4] == "left" and dataType == "left":
            with Image.open(root_path + filename) as img:
                X.append(transform(img.convert('RGB')))
            angles.append(((float(radians) * 180.0)/np.pi))
            torques.append(torque)
            speeds.append(speed)
            i += 1
            
        if filename[0:5] == "right" and dataType == "right":
            with Image.open(root_path + filename) as img:
                X.append(transform(img.convert('RGB')))
            angles.append(((float(radians) * 180.0)/np.pi))
            torques.append(torque)
            speeds.append(speed)
            i += 1
                
        if filename[0:6] == "center" and dataType == "center":
            with Image.open(root_path + filename) as img:
                X.append(transform(img.convert('RGB')))
            angles.append((float(radians) * 180.0)/np.pi)
            torques.append(torque)
            speeds.append(speed)
            i += 1
            
        if i == numDataPointsWanted: break
    end = time.time()
    

print("Data loading complete, took", int(end - start), "seconds")
print('# Images loaded: ', len(X))

Data loading complete, took 1 seconds
# Images loaded:  300


# Feature Dataset

In [18]:
class Dataset_Model(data.Dataset):
    def __init__(self, X, Y, context_size=15):
        self.X = []
        angles = Y[:,0]
        torques = Y[:,1]
        speeds = Y[:,2]
        self.angles = []
        self.torques = []
        self.speeds = []
        for i in range(context_size, len(X)):
            x = [torch.as_tensor(arr) for arr in X[i-context_size:i]]
            self.X.append(torch.stack(x))
            self.angles.append(angles[i])
            self.torques.append(torques[i])
            self.speeds.append(speeds[i])

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index].float(), self.Y[index], self.torques[index], self.speeds[index]

In [19]:
Y = np.stack([np.array(angles), np.array(torques), np.array(speeds)]).T
Y.shape

(300, 3)

In [1]:
BATCH_SIZE = 16
num_workers = 0 if cuda else 0

train_X, val_X, train_Y, val_Y = train_test_split(X, Y, test_size=0.20)
print(train_Y.shape)

train_set = Dataset_Model(train_X, train_Y, context_size=15)
train_dataloader = data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers)
val_set = Dataset_Model(val_X, val_Y, context_size=15)
val_dataloader = data.DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers)

NameError: name 'cuda' is not defined

In [13]:
print(len(train_set))
print(len(val_set))

print(train_set.__getitem__(0)[0].shape, train_set.__getitem__(0)[1])

0


NameError: name 'val_set' is not defined

# CNN model

In [17]:
class CNN(nn.Module):
    def __init__(self, in_channels, out_channels, input_size, output_size):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv3d(in_channels, out_channels, kernel_size=(1, 4, 4), stride=2, padding=2)
        self.bn1 = nn.BatchNorm3d(out_channels)
        self.relu1 = nn.ReLU()
        
        self.conv2 = nn.Conv3d(out_channels, out_channels, kernel_size=(3, 12, 12), stride=4, padding=2)
        self.bn2 = nn.BatchNorm3d(out_channels)
        self.relu2 = nn.ReLU()
        
        self.conv3 = nn.Conv3d(out_channels, out_channels, kernel_size=(1, 4, 4), stride=2, padding=2)
        self.bn3 = nn.BatchNorm3d(out_channels)
        self.relu3 = nn.ReLU()
        
        self.avgpool = nn.AvgPool3d(1)
        
#         self.convLSTM = ConvLSTM(input_size=(30, 40), input_dim=out_channels, 
#                                     hidden_dim=out_channels//2, kernel_size=(3, 3), num_layers=1, batch_first=True)
        
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, x):
        output = self.conv1(x) 
        output = self.bn1(output)
        output = self.relu1(output)
        
        output = self.conv2(output)
        output = self.bn2(output)
        output = self.relu2(output)
        
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
        
        output = self.avgpool(output)
        
#         print(output.shape)
        hidden = None
#         output, hidden = self.convLSTM(output, hidden)
        
        featuresV = output.reshape(output.size(0), -1)
        output = self.fc(featuresV)
        return output

In [18]:
context_size = 15
out_channels = 13
input_size = 46800
f_model = CNN(context_size, out_channels, input_size, 1).to(device)
optimizer = torch.optim.Adam(f_model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=1)
criterion = nn.MSELoss()

print(f_model)

CNN(
  (conv1): Conv3d(15, 13, kernel_size=(1, 4, 4), stride=(2, 2, 2), padding=(2, 2, 2))
  (bn1): BatchNorm3d(13, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (conv2): Conv3d(13, 13, kernel_size=(3, 12, 12), stride=(4, 4, 4), padding=(2, 2, 2))
  (bn2): BatchNorm3d(13, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (conv3): Conv3d(13, 13, kernel_size=(1, 4, 4), stride=(2, 2, 2), padding=(2, 2, 2))
  (bn3): BatchNorm3d(13, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (avgpool): AvgPool3d(kernel_size=1, stride=1, padding=0)
  (convLSTM): ConvLSTM(
    (cell_list): ModuleList(
      (0): ConvLSTMCell(
        (conv): Conv2d(19, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
  )
  (fc): Linear(in_features=46800, out_features=1, bias=True)
)


In [19]:
def combine(x, y):
    total = 0
    for v in range(len(x)):
        if x[v] and y[v]:
            total += 1
    return total

In [20]:
def train(num_epochs, model, save):
    EPOCH_TRAIN_LOSSES = []
    EPOCH_VAL_LOSSES = []
    EPOCH_TRAIN_ACC = []
    EPOCH_VAL_ACC = []
    train_a_acc = []
    val_a_acc = []
    
    for epoch in range(num_epochs):
        start = time.time()

        running_train_loss = 0.0
        running_val_loss = 0.0
        train_correct = 0.0
        val_correct = 0.0
        train_total = 0.0
        val_total = 0.0
               
        t_correct_predictions_0_5 = 0.0
        t_correct_predictions_1 = 0.0
        t_correct_predictions_2 = 0.0
        t_correct_predictions_5 = 0.0
        t_correct_predictions_g_5 = 0.0
        
        v_correct_predictions_0_5 = 0.0
        v_correct_predictions_1 = 0.0
        v_correct_predictions_2 = 0.0
        v_correct_predictions_5 = 0.0
        v_correct_predictions_g_5 = 0.0

        model.train()
        for i, (x, label) in enumerate(train_dataloader):
            x, label = x.to(device), label.to(device)
            optimizer.zero_grad()
#             s = time.time()
            y = model(x.float()).double().squeeze(1)
#             e = time.time()
#             print(e - s, "time for f pass")
            
            loss = criterion(y, label)
            loss.backward()
            optimizer.step()

            running_train_loss += loss.item()
            predicted = y            
            train_correct += torch.sum(torch.abs(predicted - label) < 0.1).item()
            t_correct_predictions_0_5 += torch.sum(torch.abs(predicted - label) <= 0.5).item()
            t_correct_predictions_1 += combine(torch.abs(predicted - label) > 0.5,  torch.abs(predicted - label) <= 1)
            t_correct_predictions_2 += combine(torch.abs(predicted - label) > 1,  torch.abs(predicted - label) <= 2)
            t_correct_predictions_5 += combine(torch.abs(predicted - label) > 2,  torch.abs(predicted - label) <= 5)
            v_correct_predictions_g_5 += torch.sum(torch.abs(predicted - label) > 5).item()
            train_total += label.size(0)
            
            del x
            del label
                    
        train_a_a = [t_correct_predictions_0_5, t_correct_predictions_1, t_correct_predictions_2, t_correct_predictions_5, t_correct_predictions_g_5]
        train_a_acc.append(train_a_a)

        model.eval()
        with torch.no_grad():
            for i, (x, label) in enumerate(val_dataloader):
                x, label = x.to(device), label.to(device)
                y = model(x.float()).double().squeeze(1)
                
                loss = criterion(y, label)

                running_val_loss += loss.item()
                predicted = y
                val_correct += torch.sum(torch.abs(predicted - label) < 0.1).item()
                v_correct_predictions_0_5 += torch.sum(torch.abs(predicted - label) <= 0.5).item()
                v_correct_predictions_1 += combine(torch.abs(predicted - label) > 0.5,  torch.abs(predicted - label) <= 1)
                v_correct_predictions_2 += combine(torch.abs(predicted - label) > 1,  torch.abs(predicted - label) <= 2)
                v_correct_predictions_5 += combine(torch.abs(predicted - label) > 2,  torch.abs(predicted - label) <= 5)
                v_correct_predictions_g_5 += torch.sum(torch.abs(predicted - label) > 5).item()
                val_total += label.size(0)
                
            val_a_a = [v_correct_predictions_0_5, v_correct_predictions_1, v_correct_predictions_2, v_correct_predictions_5, v_correct_predictions_g_5]
            val_a_acc.append(val_a_a)
                
        train_acc = (train_correct / (train_total))*100
        val_acc = (val_correct / (val_total))*100
        tloss = running_train_loss / len(train_dataloader)
        vloss = running_val_loss/ len(val_dataloader)

        print("Epoch", epoch, " Took", int(time.time() - start), "s")
        print("Train Acc:", train_acc, "Val Acc:", val_acc)
        print("Avg Train Loss:", tloss, "Avg Val Loss:", vloss)

        scheduler.step(vloss)

        EPOCH_TRAIN_LOSSES.append(tloss)
        EPOCH_VAL_LOSSES.append(vloss)
        EPOCH_TRAIN_ACC.append(train_acc)
        EPOCH_VAL_ACC.append(val_acc)

        if save:
            torch.save(model.state_dict(), './model_' + str(epoch + 1) + '_' + str(val_acc) + '.pt')
            
    return EPOCH_TRAIN_LOSSES, EPOCH_VAL_LOSSES, EPOCH_TRAIN_ACC, EPOCH_VAL_ACC, val_a_acc, train_a_acc

In [ ]:
n_epochs = 9
t_l, v_l, t_a, v_a, val_a_acc, train_a_acc = train(n_epochs, f_model, save=False)

In [ ]:
import matplotlib.pyplot as plt

objects = ('[0, 0.5]', '(0.5, 1]', '(1, 2]', '(2, 5]', '(5, Inf.)')
performanceV = val_a_acc[-1]
performanceT = train_a_acc[0]

x = np.arange(len(objects))
# print(v_all_a)
width = 0.2 # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, performanceV, width, label="Validation")
rects1 = ax.bar(x + width/2, performanceT, width, label="Training")

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Training Points')
ax.set_title('Accuracy for Last Epoch')
ax.set_xticks(x)
plt.ylim(0, 100)
labels = []
totalV = sum(performanceV)
totalT = sum(performanceT)

for x in range(len(objects)):
    labels.append(objects[x] + "\n" + str(int((performanceV[x]/totalV)*100)) + "%," + str(int((performanceT[x]/totalT)*100)) + "%")
ax.set_xticklabels(labels)
ax.legend()
plt.savefig('Last_Epoch.png', dpi=300)

plt.show()

objects = ('[0, 0.5]', '(0.5, 1]', '(1, 2]', '(2, 5]', '(5, Inf.)')
performanceV = val_a_acc[0]
performanceT = train_a_acc[-1]

x = np.arange(len(objects))
# print(v_all_a)
width = 0.2  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, performanceV, width, label="Validation")
rects1 = ax.bar(x + width/2, performanceT, width, label="Training")

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Training Points')
ax.set_title('Accuracy for First Epoch')
ax.set_xticks(x)
plt.ylim(0, 100)
labels = []
totalV = sum(performanceV)
totalT = sum(performanceT)

for x in range(len(objects)):
    labels.append(objects[x] + "\n" + str(int((performanceV[x]/totalV)*100)) + "%," + str(int((performanceT[x]/totalT)*100)) + "%")
ax.set_xticklabels(labels)
ax.legend()
plt.savefig('Zero_Epoch.png', dpi=300)

plt.show()

x = np.arange(n_epochs)

rangeVi = 0
rangeVf = 9
plt.plot(x[rangeVi:rangeVf], t_l[rangeVi:rangeVf], label="Train")
plt.plot(x[rangeVi:rangeVf], v_l[rangeVi:rangeVf], label="Validation")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.title("Epochs vs Loss")
plt.legend()
plt.savefig('CNN3dLossFinal.png')
plt.show()